Import

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.signal import detrend
import sys

Read

In [2]:
import os, glob

In [3]:
#_fc

# Define relative path to folder containing the text files

# files_folder = "/Users/cyrilleetude/Desktop/Internship_deliverables/datasets/Static/Healthy_static/Csv/processed"
files_folder_fc = "/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Processed_fc"#too lazy to change everything so let's keep the processed_file name for the raw ones and only change chaen saving
# files_folder= "/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/FallersCOMraw"
processed_files_fc = []

# Create a dataframe list by using a list comprehension

processed_files_fc = [pd.read_csv(file) for file in glob.glob(os.path.join(files_folder_fc ,"*.csv"))]


Time_fc= np.linspace(0,30,len(processed_files_fc[0]))
def clean_df(df):
    '''
        Clean a df by removing the firs row
    '''
    return df.loc[:, ['COM_AP', 'COM_ML']]
CleanProcessedFiles_fc= []
for i in processed_files_fc:
    CleanProcessedFiles_fc.append(clean_df(i))


def get_com(processed_df):
    # processed_df['time']= [i/100 for i in range(len(processed_df))]
    # freq= np.mean(1/np.diff(Time.to_list()))
    COM= processed_df.loc[:, ['COM_AP', 'COM_ML']]
    # COM= (COM.values)*10 #convert cm to mm no keep it in meters
    COM = detrend(COM, axis=0, type='linear')
    return COM

COM_files_fc=[]
for i in CleanProcessedFiles_fc:
    COM_files_fc.append(get_com(i))



#function
def hyperellipsoid(P, y=None, z=None, pvalue=.95, units=None, show=False, ax=None):
    """
    Prediction hyperellipsoid for multivariate data.

    The hyperellipsoid is a prediction interval for a sample of a multivariate
    random variable and is such that there is pvalue*100% of probability that a
    new observation will be contained inside the hyperellipsoid [1]_.  
    The hyperellipsoid is also a tolerance region such that the average or
    expected value of the proportion of the population contained in this region
    is exactly pvalue*100% (called Type 2 tolerance region by Chew (1966) [1]_).

    The directions and lengths of the semi-axes are found, respectively, as the
    eigenvectors and eigenvalues of the covariance matrix of the data using
    the concept of principal components analysis (PCA) [2]_ or singular value
    decomposition (SVD) [3]_ and the length of the semi-axes are adjusted to
    account for the necessary prediction probability.

    The volume of the hyperellipsoid is calculated with the same equation for
    the volume of a n-dimensional ball [4]_ with the radius replaced by the
    semi-axes of the hyperellipsoid.

    This function calculates the prediction hyperellipsoid for the data,
    which is considered a (finite) sample of a multivariate random variable
    with normal distribution (i.e., the F distribution is used and not
    the approximation by the chi-square distribution).

    Parameters
    ----------
    P : 1-D or 2-D array_like
        For a 1-D array, P is the abscissa values of the [x,y] or [x,y,z] data.
        For a 2-D array, P is the joined values of the multivariate data.
        The shape of the 2-D array should be (n, p) where n is the number of
        observations (rows) and p the number of dimensions (columns).
    y : 1-D array_like, optional (default = None)
        Ordinate values of the [x, y] or [x, y, z] data.
    z : 1-D array_like, optional (default = None)
        Ordinate values of the [x, y] or [x, y, z] data.
    pvalue : float, optional (default = .95)
        Desired prediction probability of the hyperellipsoid.
    units : str, optional (default = None)
        Units of the input data.
    show : bool, optional (default = True)
        True (1) plots data in a matplotlib figure, False (0) to not plot.
        Only the results for p=2 (ellipse) or p=3 (ellipsoid) will be plotted.
    ax : a matplotlib.axes.Axes instance (default = None)

    Returns
    -------
    hypervolume : float
        Hypervolume (e.g., area of the ellipse or volume of the ellipsoid).
    axes : 1-D array
        Lengths of the semi-axes hyperellipsoid (largest first).
    angles : 1-D array
        Angles of the semi-axes hyperellipsoid (only for 2D or 3D data).
        For the ellipsoid (3D data), the angles are the Euler angles
        calculated in the XYZ sequence.
    center : 1-D array
        Centroid of the hyperellipsoid.
    rotation : 2-D array
        Rotation matrix for hyperellipsoid semi-axes (only for 2D or 3D data).

    References
    ----------
    .. [1] http://www.jstor.org/stable/2282774
    .. [2] http://en.wikipedia.org/wiki/Principal_component_analysis
    .. [3] http://en.wikipedia.org/wiki/Singular_value_decomposition
    .. [4] http://en.wikipedia.org/wiki/Volume_of_an_n-ball

    Examples
    --------
    >>> from hyperellipsoid import hyperellipsoid
    >>> y = np.cumsum(np.random.randn(3000)) / 50
    >>> x = np.cumsum(np.random.randn(3000)) / 100
    >>> area, axes, angles, center, R = hyperellipsoid(x, y, units='cm')
    >>> print('Area =', area)
    >>> print('Semi-axes =', axes)
    >>> print('Angles =', angles)
    >>> print('Center =', center)
    >>> print('Rotation matrix =\n', R)

    >>> P = np.random.randn(1000, 3)
    >>> P[:, 2] = P[:, 2] + P[:, 1]*.5
    >>> P[:, 1] = P[:, 1] + P[:, 0]*.5
    >>> volume, axes, angles, center, R = hyperellipsoid(P, units='cm')
    """

    from scipy.stats import f as F
    from scipy.special import gamma

    P = np.array(P, ndmin=2, dtype=float)
    if P.shape[0] == 1:
        P = P.T
    if y is not None:
        y = np.array(y, copy=False, ndmin=2, dtype=float)
        if y.shape[0] == 1:
            y = y.T
        P = np.concatenate((P, y), axis=1)
    if z is not None:
        z = np.array(z, copy=False, ndmin=2, dtype=float)
        if z.shape[0] == 1:
            z = z.T
        P = np.concatenate((P, z), axis=1)
    # covariance matrix
    cov = np.cov(P, rowvar=0)
    # singular value decomposition
    U, s, Vt = np.linalg.svd(cov)
    p, n = s.size, P.shape[0]
    # F percent point function
    fppf = F.ppf(pvalue, p, n-p)*(n-1)*p*(n+1)/n/(n-p)
    # semi-axes (largest first)
    saxes = np.sqrt(s*fppf)
    hypervolume = np.pi**(p/2)/gamma(p/2+1)*np.prod(saxes)
    # rotation matrix
    if p == 2 or p == 3:
        R = Vt
        if s.size == 2:
            angles = np.array([np.rad2deg(np.arctan2(R[1, 0], R[0, 0])),
                               90-np.rad2deg(np.arctan2(R[1, 0], -R[0, 0]))])
        else:
            angles = rotXYZ(R, unit='deg')
        # centroid of the hyperellipsoid
        center = np.mean(P, axis=0)
    else:
        R, angles = None, None

    if show and (p == 2 or p == 3):
        _plot(P, hypervolume, saxes, center, R, pvalue, units, ax)

    return hypervolume, saxes, angles, center, R


def _plot(P, hypervolume, saxes, center, R, pvalue, units, ax):
    """Plot results of the hyperellipsoid function, see its help."""

    try:
        import matplotlib.pyplot as plt
    except ImportError:
        print('matplotlib is not available.')
    else:
        # code based on https://github.com/minillinim/ellipsoid:
        # parametric equations
        u = np.linspace(0, 2*np.pi, 100)
        if saxes.size == 2:
            x = saxes[0]*np.cos(u)
            y = saxes[1]*np.sin(u)
            # rotate data
            for i in range(len(x)):
                [x[i], y[i]] = np.dot([x[i], y[i]], R) + center
        else:
            v = np.linspace(0, np.pi, 100)
            x = saxes[0]*np.outer(np.cos(u), np.sin(v))
            y = saxes[1]*np.outer(np.sin(u), np.sin(v))
            z = saxes[2]*np.outer(np.ones_like(u), np.cos(v))
            # rotate data
            for i in range(len(x)):
                for j in range(len(x)):
                    [x[i,j],y[i,j],z[i,j]] = np.dot([x[i,j],y[i,j],z[i,j]], R) + center

        if saxes.size == 2:
            if ax is None:
                fig, ax = plt.subplots(1, 1, figsize=(5, 5))
            # plot raw data
            ax.plot(P[:, 0], P[:, 1], '.-', color=[0, 0, 1, .5])
            # plot ellipse
            ax.plot(x, y, color=[0, 1, 0, 1], linewidth=2)
            # plot axes
            for i in range(saxes.size):
                # rotate axes
                a = np.dot(np.diag(saxes)[i], R).reshape(2, 1)
                # points for the axes extremities
                a = np.dot(a, np.array([-1, 1], ndmin=2))+center.reshape(2, 1)
                ax.plot(a[0], a[1], color=[1, 0, 0, .6], linewidth=2)
                ax.text(a[0, 1], a[1, 1], '%d' % (i + 1),
                        fontsize=20, color='r')
            plt.axis('equal')
            plt.grid()
            title = r'Prediction ellipse (p=%4.2f): Area=' % pvalue
            if units is not None:
                units2 = ' [%s]' % units
                units = units + r'$^2$'
                title = title + r'%.2f %s' % (hypervolume, units)
            else:
                units2 = ''
                title = title + r'%.2f' % hypervolume
        else:
            from mpl_toolkits.mplot3d import Axes3D
            if ax is None:
                fig = plt.figure(figsize=(7, 7))
                ax = fig.add_axes([0, 0, 1, 1], projection='3d')
            ax.view_init(20, 30)
            # plot raw data
            ax.plot(P[:, 0], P[:, 1], P[:, 2], '.-', color=[0, 0, 1, .4])
            # plot ellipsoid
            ax.plot_surface(x, y, z, rstride=5, cstride=5, color=[0, 1, 0, .1],
                            linewidth=1, edgecolor=[.1, .9, .1, .4])
            # ax.plot_wireframe(x, y, z, color=[0, 1, 0, .5], linewidth=1)
            #                  rstride=3, cstride=3, edgecolor=[0, 1, 0, .5])
            # plot axes
            for i in range(saxes.size):
                # rotate axes
                a = np.dot(np.diag(saxes)[i], R).reshape(3, 1)
                # points for the axes extremities
                a = np.dot(a, np.array([-1, 1], ndmin=2))+center.reshape(3, 1)
                ax.plot(a[0], a[1], a[2], color=[1, 0, 0, .6], linewidth=2)
                ax.text(a[0, 1], a[1, 1], a[2, 1], '%d' % (i+1),
                        fontsize=20, color='r')
            lims = [np.min([P.min(), x.min(), y.min(), z.min()]),
                    np.max([P.max(), x.max(), y.max(), z.max()])]
            ax.set_xlim(lims)
            ax.set_ylim(lims)
            ax.set_zlim(lims)
            title = r'Prediction ellipsoid (p=%4.2f): Volume=' % pvalue
            if units is not None:
                units2 = ' [%s]' % units
                units = units + r'$^3$'
                title = title + r'%.2f %s' % (hypervolume, units)
            else:
                units2 = ''
                title = title + r'%.2f' % hypervolume
            ax.set_zlabel('Z' + units2, fontsize=18)

        ax.set_xlabel('X' + units2, fontsize=18)
        ax.set_ylabel('Y' + units2, fontsize=18)
        plt.title(title)
        plt.show()

        return ax


def rotXYZ(R, unit='deg'):
    """ Compute Euler angles from matrix R using XYZ sequence."""

    angles = np.zeros(3)
    angles[0] = np.arctan2(R[2, 1], R[2, 2])
    angles[1] = np.arctan2(-R[2, 0], np.sqrt(R[0, 0]**2 + R[1, 0]**2))
    angles[2] = np.arctan2(R[1, 0], R[0, 0])

    if unit[:3].lower() == 'deg':  # convert from rad to degree
        angles = np.rad2deg(angles)

    return angles




#sway area
def get_area(COM_data):
    area, axes, angles, center, R = hyperellipsoid((COM_data)[:, 1], (COM_data)[:, 0], units='m')
    return area
Areas_fc=[]
for i in COM_files_fc:
    Areas_fc.append(get_area(i))


#Range AP
def rang(COM, axis_id):
    rang = np.max(COM, axis=0) - np.min(COM, axis=0)  # range (maximum - minimum)
    return rang[axis_id]

def get_rang_AP(COM_data):
    range_ap= rang(COM_data, 0)
    return range_ap

range_AP_fc=[]
for i in COM_files_fc:
    range_AP_fc.append(get_rang_AP(i))

#Range ML
def get_rang_ML(COM_data):
    range_ml= rang(COM_data, 1)
    return range_ml

range_ML_fc=[]
for i in COM_files_fc:
    range_ML_fc.append(get_rang_ML(i))




#Sway Length AP
def sway_len(COM, axis_id):
    tpath = np.sum(np.abs(np.diff(COM, axis=0)), axis=0) # total path (length of the COP displacement)
    return tpath[axis_id]

def get_SL_AP(COM_data):
    sl_ap= sway_len(COM_data, 0)
    return sl_ap

SL_AP_fc=[]
for i in COM_files_fc:
    SL_AP_fc.append(get_SL_AP(i))

#Sl ML
def get_SL_ML(COM_data):
    sl_ml= sway_len(COM_data, 1)
    return sl_ml

SL_ML_fc=[]
for i in COM_files_fc:
    SL_ML_fc.append(get_SL_ML(i))


# Velocity
def get_Velocity(disp_array, time_array):
    '''
        Calculate the velocity of COM
    '''
    # disp_array= df.values
    x= (np.abs(np.diff(disp_array[:, 0])))/ np.diff(time_array)#ap
    y= (np.abs(np.diff(disp_array[:, 1])))/ np.diff(time_array)#ml
    df = pd.DataFrame({ 'Vel_COM_AP':x,'Vel_COM_ML':y})
    return df.values
    # Data= Data.apply(lambda x: x/1000)
velocities_fc=[]
for i in COM_files_fc:
    velocities_fc.append(get_Velocity(i, Time_fc))
# Mean Velocity
def get_mean_Velocity(disp_array, time_array):
    '''
        Calculate the mean velocity of COM
    '''
    x= np.sum(np.abs(np.diff(disp_array[:, 0])))/ np.sum(np.diff(time_array))#ap
    y= np.sum(np.abs(np.diff(disp_array[:, 1])))/ np.sum(np.diff(time_array))#ml
    # df = pd.DataFrame({ 'Mean_Vel_COM_AP':x,'Mean_Vel_COM_ML':y})
    return np.array([x,y])
mean_velocities_fc=[]
for i in COM_files_fc:
    mean_velocities_fc.append(get_mean_Velocity(i, Time_fc))



# acceleration
Time_fc= np.linspace(0,30,len(velocities_fc[0]))#update time
def get_acceleration(vel_array, time_array):
    '''
        Calculate the acceleration of COM
    '''
    # vel_array= df.values
    x= (np.diff(vel_array[:, 0]))/ np.diff(time_array)#ap
    y= (np.diff(vel_array[:, 1]))/ np.diff(time_array)#ml
    df = pd.DataFrame({ 'Acc_COM_AP':x,'Acc_COM_ML':y})
    return df.values
    # Data= Data.apply(lambda x: x/1000)
acc_fc=[]
for i in velocities_fc:
    acc_fc.append(get_acceleration(i, Time_fc))
# Mean acceleration
def get_mean_acceleration(vel_array, time_array):
    '''
        Calculate the mean acceleration of COM
    '''
    x= np.sum(np.abs(np.diff(vel_array[:, 0])))/ np.sum(np.diff(time_array))#ap
    y= np.sum(np.abs(np.diff(vel_array[:, 1])))/ np.sum(np.diff(time_array))#ml
    # df = pd.DataFrame({ 'Mean_Vel_COM_AP':x,'Mean_Vel_COM_ML':y})
    return np.array([x,y])
mean_acc_fc=[]
for i in velocities_fc:
    mean_acc_fc.append(get_mean_acceleration(i, Time_fc))
def get_rms_acceleration(mean_acc_list):
    '''
        Calculate the rms acceleration of COM
    '''
    rms_acceleration=[]
    for i in mean_acc_list:
        rms_acceleration.append(np.sqrt(i))
    return rms_acceleration
rms_acc_fc= get_rms_acceleration(mean_acc_fc)







data_fc = []
for i in range(len(COM_files_fc)):
    outcomes=[Areas_fc[i], range_AP_fc[i], range_ML_fc[i], SL_AP_fc[i], SL_ML_fc[i], mean_velocities_fc[i][0], mean_velocities_fc[i][1]
    , (rms_acc_fc[i]/9.8)[0], (rms_acc_fc[i]/9.8)[1]]#convert in g-unit
    data_fc.append(outcomes)

df_fc = pd.DataFrame(data_fc, columns=['Area (m^2)_fc', 'Range AP (m)_fc', 'Range ML (m)_fc', "Sway Length AP (m)_fc", "Sway Length ML (m)_fc",
'Mean Velocity AP (m/s)_fc', 'Mean Velocity ML (m/s)_fc', 'RMS Acc AP (g)_fc', 'RMS Acc ML (g)_fc'])



In [4]:
#_fo

# Define relative path to folder containing the text files

# files_folder = "/Users/cyrilleetude/Desktop/Internship_deliverables/datasets/Static/Healthy_static/Csv/processed"
files_folder_fo = "/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Processed_fo"#too lazy to change everything so let's keep the processed_file name for the raw ones and only change chaen saving
# files_folder= "/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/FallersCOMraw"
processed_files_fo = []

# Create a dataframe list by using a list comprehension

processed_files_fo = [pd.read_csv(file) for file in glob.glob(os.path.join(files_folder_fo ,"*.csv"))]


Time_fo= np.linspace(0,30,len(processed_files_fo[0]))
def clean_df(df):
    '''
        Clean a df by removing the firs row
    '''
    return df.loc[:, ['COM_AP', 'COM_ML']]
CleanProcessedFiles_fo= []
for i in processed_files_fo:
    CleanProcessedFiles_fo.append(clean_df(i))


def get_com(processed_df):
    # processed_df['time']= [i/100 for i in range(len(processed_df))]
    # freq= np.mean(1/np.diff(Time.to_list()))
    COM= processed_df.loc[:, ['COM_AP', 'COM_ML']]
    # COM= (COM.values)*10 #convert cm to mm no keep it in meters
    COM = detrend(COM, axis=0, type='linear')
    return COM

COM_files_fo=[]
for i in CleanProcessedFiles_fo:
    COM_files_fo.append(get_com(i))



#function
def hyperellipsoid(P, y=None, z=None, pvalue=.95, units=None, show=False, ax=None):
    """
    Prediction hyperellipsoid for multivariate data.

    The hyperellipsoid is a prediction interval for a sample of a multivariate
    random variable and is such that there is pvalue*100% of probability that a
    new observation will be contained inside the hyperellipsoid [1]_.  
    The hyperellipsoid is also a tolerance region such that the average or
    expected value of the proportion of the population contained in this region
    is exactly pvalue*100% (called Type 2 tolerance region by Chew (1966) [1]_).

    The directions and lengths of the semi-axes are found, respectively, as the
    eigenvectors and eigenvalues of the covariance matrix of the data using
    the concept of principal components analysis (PCA) [2]_ or singular value
    decomposition (SVD) [3]_ and the length of the semi-axes are adjusted to
    account for the necessary prediction probability.

    The volume of the hyperellipsoid is calculated with the same equation for
    the volume of a n-dimensional ball [4]_ with the radius replaced by the
    semi-axes of the hyperellipsoid.

    This function calculates the prediction hyperellipsoid for the data,
    which is considered a (finite) sample of a multivariate random variable
    with normal distribution (i.e., the F distribution is used and not
    the approximation by the chi-square distribution).

    Parameters
    ----------
    P : 1-D or 2-D array_like
        For a 1-D array, P is the abscissa values of the [x,y] or [x,y,z] data.
        For a 2-D array, P is the joined values of the multivariate data.
        The shape of the 2-D array should be (n, p) where n is the number of
        observations (rows) and p the number of dimensions (columns).
    y : 1-D array_like, optional (default = None)
        Ordinate values of the [x, y] or [x, y, z] data.
    z : 1-D array_like, optional (default = None)
        Ordinate values of the [x, y] or [x, y, z] data.
    pvalue : float, optional (default = .95)
        Desired prediction probability of the hyperellipsoid.
    units : str, optional (default = None)
        Units of the input data.
    show : bool, optional (default = True)
        True (1) plots data in a matplotlib figure, False (0) to not plot.
        Only the results for p=2 (ellipse) or p=3 (ellipsoid) will be plotted.
    ax : a matplotlib.axes.Axes instance (default = None)

    Returns
    -------
    hypervolume : float
        Hypervolume (e.g., area of the ellipse or volume of the ellipsoid).
    axes : 1-D array
        Lengths of the semi-axes hyperellipsoid (largest first).
    angles : 1-D array
        Angles of the semi-axes hyperellipsoid (only for 2D or 3D data).
        For the ellipsoid (3D data), the angles are the Euler angles
        calculated in the XYZ sequence.
    center : 1-D array
        Centroid of the hyperellipsoid.
    rotation : 2-D array
        Rotation matrix for hyperellipsoid semi-axes (only for 2D or 3D data).

    References
    ----------
    .. [1] http://www.jstor.org/stable/2282774
    .. [2] http://en.wikipedia.org/wiki/Principal_component_analysis
    .. [3] http://en.wikipedia.org/wiki/Singular_value_decomposition
    .. [4] http://en.wikipedia.org/wiki/Volume_of_an_n-ball

    Examples
    --------
    >>> from hyperellipsoid import hyperellipsoid
    >>> y = np.cumsum(np.random.randn(3000)) / 50
    >>> x = np.cumsum(np.random.randn(3000)) / 100
    >>> area, axes, angles, center, R = hyperellipsoid(x, y, units='cm')
    >>> print('Area =', area)
    >>> print('Semi-axes =', axes)
    >>> print('Angles =', angles)
    >>> print('Center =', center)
    >>> print('Rotation matrix =\n', R)

    >>> P = np.random.randn(1000, 3)
    >>> P[:, 2] = P[:, 2] + P[:, 1]*.5
    >>> P[:, 1] = P[:, 1] + P[:, 0]*.5
    >>> volume, axes, angles, center, R = hyperellipsoid(P, units='cm')
    """

    from scipy.stats import f as F
    from scipy.special import gamma

    P = np.array(P, ndmin=2, dtype=float)
    if P.shape[0] == 1:
        P = P.T
    if y is not None:
        y = np.array(y, copy=False, ndmin=2, dtype=float)
        if y.shape[0] == 1:
            y = y.T
        P = np.concatenate((P, y), axis=1)
    if z is not None:
        z = np.array(z, copy=False, ndmin=2, dtype=float)
        if z.shape[0] == 1:
            z = z.T
        P = np.concatenate((P, z), axis=1)
    # covariance matrix
    cov = np.cov(P, rowvar=0)
    # singular value decomposition
    U, s, Vt = np.linalg.svd(cov)
    p, n = s.size, P.shape[0]
    # F percent point function
    fppf = F.ppf(pvalue, p, n-p)*(n-1)*p*(n+1)/n/(n-p)
    # semi-axes (largest first)
    saxes = np.sqrt(s*fppf)
    hypervolume = np.pi**(p/2)/gamma(p/2+1)*np.prod(saxes)
    # rotation matrix
    if p == 2 or p == 3:
        R = Vt
        if s.size == 2:
            angles = np.array([np.rad2deg(np.arctan2(R[1, 0], R[0, 0])),
                               90-np.rad2deg(np.arctan2(R[1, 0], -R[0, 0]))])
        else:
            angles = rotXYZ(R, unit='deg')
        # centroid of the hyperellipsoid
        center = np.mean(P, axis=0)
    else:
        R, angles = None, None

    if show and (p == 2 or p == 3):
        _plot(P, hypervolume, saxes, center, R, pvalue, units, ax)

    return hypervolume, saxes, angles, center, R


def _plot(P, hypervolume, saxes, center, R, pvalue, units, ax):
    """Plot results of the hyperellipsoid function, see its help."""

    try:
        import matplotlib.pyplot as plt
    except ImportError:
        print('matplotlib is not available.')
    else:
        # code based on https://github.com/minillinim/ellipsoid:
        # parametric equations
        u = np.linspace(0, 2*np.pi, 100)
        if saxes.size == 2:
            x = saxes[0]*np.cos(u)
            y = saxes[1]*np.sin(u)
            # rotate data
            for i in range(len(x)):
                [x[i], y[i]] = np.dot([x[i], y[i]], R) + center
        else:
            v = np.linspace(0, np.pi, 100)
            x = saxes[0]*np.outer(np.cos(u), np.sin(v))
            y = saxes[1]*np.outer(np.sin(u), np.sin(v))
            z = saxes[2]*np.outer(np.ones_like(u), np.cos(v))
            # rotate data
            for i in range(len(x)):
                for j in range(len(x)):
                    [x[i,j],y[i,j],z[i,j]] = np.dot([x[i,j],y[i,j],z[i,j]], R) + center

        if saxes.size == 2:
            if ax is None:
                fig, ax = plt.subplots(1, 1, figsize=(5, 5))
            # plot raw data
            ax.plot(P[:, 0], P[:, 1], '.-', color=[0, 0, 1, .5])
            # plot ellipse
            ax.plot(x, y, color=[0, 1, 0, 1], linewidth=2)
            # plot axes
            for i in range(saxes.size):
                # rotate axes
                a = np.dot(np.diag(saxes)[i], R).reshape(2, 1)
                # points for the axes extremities
                a = np.dot(a, np.array([-1, 1], ndmin=2))+center.reshape(2, 1)
                ax.plot(a[0], a[1], color=[1, 0, 0, .6], linewidth=2)
                ax.text(a[0, 1], a[1, 1], '%d' % (i + 1),
                        fontsize=20, color='r')
            plt.axis('equal')
            plt.grid()
            title = r'Prediction ellipse (p=%4.2f): Area=' % pvalue
            if units is not None:
                units2 = ' [%s]' % units
                units = units + r'$^2$'
                title = title + r'%.2f %s' % (hypervolume, units)
            else:
                units2 = ''
                title = title + r'%.2f' % hypervolume
        else:
            from mpl_toolkits.mplot3d import Axes3D
            if ax is None:
                fig = plt.figure(figsize=(7, 7))
                ax = fig.add_axes([0, 0, 1, 1], projection='3d')
            ax.view_init(20, 30)
            # plot raw data
            ax.plot(P[:, 0], P[:, 1], P[:, 2], '.-', color=[0, 0, 1, .4])
            # plot ellipsoid
            ax.plot_surface(x, y, z, rstride=5, cstride=5, color=[0, 1, 0, .1],
                            linewidth=1, edgecolor=[.1, .9, .1, .4])
            # ax.plot_wireframe(x, y, z, color=[0, 1, 0, .5], linewidth=1)
            #                  rstride=3, cstride=3, edgecolor=[0, 1, 0, .5])
            # plot axes
            for i in range(saxes.size):
                # rotate axes
                a = np.dot(np.diag(saxes)[i], R).reshape(3, 1)
                # points for the axes extremities
                a = np.dot(a, np.array([-1, 1], ndmin=2))+center.reshape(3, 1)
                ax.plot(a[0], a[1], a[2], color=[1, 0, 0, .6], linewidth=2)
                ax.text(a[0, 1], a[1, 1], a[2, 1], '%d' % (i+1),
                        fontsize=20, color='r')
            lims = [np.min([P.min(), x.min(), y.min(), z.min()]),
                    np.max([P.max(), x.max(), y.max(), z.max()])]
            ax.set_xlim(lims)
            ax.set_ylim(lims)
            ax.set_zlim(lims)
            title = r'Prediction ellipsoid (p=%4.2f): Volume=' % pvalue
            if units is not None:
                units2 = ' [%s]' % units
                units = units + r'$^3$'
                title = title + r'%.2f %s' % (hypervolume, units)
            else:
                units2 = ''
                title = title + r'%.2f' % hypervolume
            ax.set_zlabel('Z' + units2, fontsize=18)

        ax.set_xlabel('X' + units2, fontsize=18)
        ax.set_ylabel('Y' + units2, fontsize=18)
        plt.title(title)
        plt.show()

        return ax


def rotXYZ(R, unit='deg'):
    """ Compute Euler angles from matrix R using XYZ sequence."""

    angles = np.zeros(3)
    angles[0] = np.arctan2(R[2, 1], R[2, 2])
    angles[1] = np.arctan2(-R[2, 0], np.sqrt(R[0, 0]**2 + R[1, 0]**2))
    angles[2] = np.arctan2(R[1, 0], R[0, 0])

    if unit[:3].lower() == 'deg':  # convert from rad to degree
        angles = np.rad2deg(angles)

    return angles




#sway area
def get_area(COM_data):
    area, axes, angles, center, R = hyperellipsoid((COM_data)[:, 1], (COM_data)[:, 0], units='m')
    return area
Areas_fo=[]
for i in COM_files_fo:
    Areas_fo.append(get_area(i))


#Range AP
def rang(COM, axis_id):
    rang = np.max(COM, axis=0) - np.min(COM, axis=0)  # range (maximum - minimum)
    return rang[axis_id]

def get_rang_AP(COM_data):
    range_ap= rang(COM_data, 0)
    return range_ap

range_AP_fo=[]
for i in COM_files_fo:
    range_AP_fo.append(get_rang_AP(i))

#Range ML
def get_rang_ML(COM_data):
    range_ml= rang(COM_data, 1)
    return range_ml

range_ML_fo=[]
for i in COM_files_fo:
    range_ML_fo.append(get_rang_ML(i))




#Sway Length AP
def sway_len(COM, axis_id):
    tpath = np.sum(np.abs(np.diff(COM, axis=0)), axis=0) # total path (length of the COP displacement)
    return tpath[axis_id]

def get_SL_AP(COM_data):
    sl_ap= sway_len(COM_data, 0)
    return sl_ap

SL_AP_fo=[]
for i in COM_files_fo:
    SL_AP_fo.append(get_SL_AP(i))

#Sl ML
def get_SL_ML(COM_data):
    sl_ml= sway_len(COM_data, 1)
    return sl_ml

SL_ML_fo=[]
for i in COM_files_fo:
    SL_ML_fo.append(get_SL_ML(i))


# Velocity
def get_Velocity(disp_array, time_array):
    '''
        Calculate the velocity of COM
    '''
    # disp_array= df.values
    x= (np.abs(np.diff(disp_array[:, 0])))/ np.diff(time_array)#ap
    y= (np.abs(np.diff(disp_array[:, 1])))/ np.diff(time_array)#ml
    df = pd.DataFrame({ 'Vel_COM_AP':x,'Vel_COM_ML':y})
    return df.values
    # Data= Data.apply(lambda x: x/1000)
velocities_fo=[]
for i in COM_files_fo:
    velocities_fo.append(get_Velocity(i, Time_fo))
# Mean Velocity
def get_mean_Velocity(disp_array, time_array):
    '''
        Calculate the mean velocity of COM
    '''
    x= np.sum(np.abs(np.diff(disp_array[:, 0])))/ np.sum(np.diff(time_array))#ap
    y= np.sum(np.abs(np.diff(disp_array[:, 1])))/ np.sum(np.diff(time_array))#ml
    # df = pd.DataFrame({ 'Mean_Vel_COM_AP':x,'Mean_Vel_COM_ML':y})
    return np.array([x,y])
mean_velocities_fo=[]
for i in COM_files_fo:
    mean_velocities_fo.append(get_mean_Velocity(i, Time_fo))



# acceleration
Time_fo= np.linspace(0,30,len(velocities_fo[0]))#update time
def get_acceleration(vel_array, time_array):
    '''
        Calculate the acceleration of COM
    '''
    # vel_array= df.values
    x= (np.diff(vel_array[:, 0]))/ np.diff(time_array)#ap
    y= (np.diff(vel_array[:, 1]))/ np.diff(time_array)#ml
    df = pd.DataFrame({ 'Acc_COM_AP':x,'Acc_COM_ML':y})
    return df.values
    # Data= Data.apply(lambda x: x/1000)
acc_fo=[]
for i in velocities_fo:
    acc_fo.append(get_acceleration(i, Time_fo))
# Mean acceleration
def get_mean_acceleration(vel_array, time_array):
    '''
        Calculate the mean acceleration of COM
    '''
    x= np.sum(np.abs(np.diff(vel_array[:, 0])))/ np.sum(np.diff(time_array))#ap
    y= np.sum(np.abs(np.diff(vel_array[:, 1])))/ np.sum(np.diff(time_array))#ml
    # df = pd.DataFrame({ 'Mean_Vel_COM_AP':x,'Mean_Vel_COM_ML':y})
    return np.array([x,y])
mean_acc_fo=[]
for i in velocities_fo:
    mean_acc_fo.append(get_mean_acceleration(i, Time_fo))
def get_rms_acceleration(mean_acc_list):
    '''
        Calculate the rms acceleration of COM
    '''
    rms_acceleration=[]
    for i in mean_acc_list:
        rms_acceleration.append(np.sqrt(i))
    return rms_acceleration
rms_acc_fo= get_rms_acceleration(mean_acc_fo)







data_fo = []
for i in range(len(COM_files_fo)):
    outcomes=[Areas_fo[i], range_AP_fo[i], range_ML_fo[i], SL_AP_fo[i], SL_ML_fo[i], mean_velocities_fo[i][0], mean_velocities_fo[i][1]
    , (rms_acc_fo[i]/9.8)[0], (rms_acc_fo[i]/9.8)[1]]#convert in g-unit
    data_fo.append(outcomes)

df_fo = pd.DataFrame(data_fo, columns=['Area (m^2)_fo', 'Range AP (m)_fo', 'Range ML (m)_fo', "Sway Length AP (m)_fo", "Sway Length ML (m)_fo",
'Mean Velocity AP (m/s)_fo', 'Mean Velocity ML (m/s)_fo', 'RMS Acc AP (g)_fo', 'RMS Acc ML (g)_fo'])
df_fo



,Area (m^2)_fo,Range AP (m)_fo,Range ML (m)_fo,Sway Length AP (m)_fo,Sway Length ML (m)_fo,Mean Velocity AP (m/s)_fo,Mean Velocity ML (m/s)_fo,RMS Acc AP (g)_fo,RMS Acc ML (g)_fo
0,0.001394,0.041658,0.032164,0.326656,0.229498,0.010889,0.007650,0.018146,0.014710
1,0.001160,0.041066,0.038875,0.290913,0.224841,0.009697,0.007495,0.018580,0.013547
2,0.000335,0.029771,0.014024,0.335877,0.155922,0.011196,0.005197,0.019420,0.013648
3,0.001377,0.039606,0.037972,0.267982,0.218399,0.008933,0.007280,0.014433,0.012557
4,0.001151,0.043211,0.048479,0.343351,0.327831,0.011445,0.010928,0.021490,0.020217
5,0.001080,0.038324,0.031304,0.232946,0.193739,0.007765,0.006458,0.014254,0.011970
6,0.000449,0.029935,0.025375,0.299782,0.240127,0.009993,0.008004,0.020318,0.018219
7,0.000994,0.040378,0.022297,0.236641,0.146829,0.007888,0.004894,0.016466,0.012359
8,0.001832,0.043477,0.043342,0.367684,0.301163,0.012256,0.010039,0.021124,0.015489
9,0.001417,0.047433,0.034816,0.379272,0.255130,0.012642,0.008504,0.019061,0.015485


In [5]:
#_gc

# Define relative path to folder containing the text files

# files_folder = "/Users/cyrilleetude/Desktop/Internship_deliverables/datasets/Static/Healthy_static/Csv/processed"
files_folder_gc = "/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Processed_gc"#too lazy to change everything so let's keep the processed_file name for the raw ones and only change chaen saving
# files_folder= "/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/FallersCOMraw"
processed_files_gc = []

# Create a dataframe list by using a list comprehension

processed_files_gc = [pd.read_csv(file) for file in glob.glob(os.path.join(files_folder_gc ,"*.csv"))]


Time_gc= np.linspace(0,30,len(processed_files_gc[0]))
def clean_df(df):
    '''
        Clean a df by removing the firs row
    '''
    return df.loc[:, ['COM_AP', 'COM_ML']]
CleanProcessedFiles_gc= []
for i in processed_files_gc:
    CleanProcessedFiles_gc.append(clean_df(i))


def get_com(processed_df):
    # processed_df['time']= [i/100 for i in range(len(processed_df))]
    # freq= np.mean(1/np.diff(Time.to_list()))
    COM= processed_df.loc[:, ['COM_AP', 'COM_ML']]
    # COM= (COM.values)*10 #convert cm to mm no keep it in meters
    COM = detrend(COM, axis=0, type='linear')
    return COM

COM_files_gc=[]
for i in CleanProcessedFiles_gc:
    COM_files_gc.append(get_com(i))



#function
def hyperellipsoid(P, y=None, z=None, pvalue=.95, units=None, show=False, ax=None):
    """
    Prediction hyperellipsoid for multivariate data.

    The hyperellipsoid is a prediction interval for a sample of a multivariate
    random variable and is such that there is pvalue*100% of probability that a
    new observation will be contained inside the hyperellipsoid [1]_.  
    The hyperellipsoid is also a tolerance region such that the average or
    expected value of the proportion of the population contained in this region
    is exactly pvalue*100% (called Type 2 tolerance region by Chew (1966) [1]_).

    The directions and lengths of the semi-axes are found, respectively, as the
    eigenvectors and eigenvalues of the covariance matrix of the data using
    the concept of principal components analysis (PCA) [2]_ or singular value
    decomposition (SVD) [3]_ and the length of the semi-axes are adjusted to
    account for the necessary prediction probability.

    The volume of the hyperellipsoid is calculated with the same equation for
    the volume of a n-dimensional ball [4]_ with the radius replaced by the
    semi-axes of the hyperellipsoid.

    This function calculates the prediction hyperellipsoid for the data,
    which is considered a (finite) sample of a multivariate random variable
    with normal distribution (i.e., the F distribution is used and not
    the approximation by the chi-square distribution).

    Parameters
    ----------
    P : 1-D or 2-D array_like
        For a 1-D array, P is the abscissa values of the [x,y] or [x,y,z] data.
        For a 2-D array, P is the joined values of the multivariate data.
        The shape of the 2-D array should be (n, p) where n is the number of
        observations (rows) and p the number of dimensions (columns).
    y : 1-D array_like, optional (default = None)
        Ordinate values of the [x, y] or [x, y, z] data.
    z : 1-D array_like, optional (default = None)
        Ordinate values of the [x, y] or [x, y, z] data.
    pvalue : float, optional (default = .95)
        Desired prediction probability of the hyperellipsoid.
    units : str, optional (default = None)
        Units of the input data.
    show : bool, optional (default = True)
        True (1) plots data in a matplotlib figure, False (0) to not plot.
        Only the results for p=2 (ellipse) or p=3 (ellipsoid) will be plotted.
    ax : a matplotlib.axes.Axes instance (default = None)

    Returns
    -------
    hypervolume : float
        Hypervolume (e.g., area of the ellipse or volume of the ellipsoid).
    axes : 1-D array
        Lengths of the semi-axes hyperellipsoid (largest first).
    angles : 1-D array
        Angles of the semi-axes hyperellipsoid (only for 2D or 3D data).
        For the ellipsoid (3D data), the angles are the Euler angles
        calculated in the XYZ sequence.
    center : 1-D array
        Centroid of the hyperellipsoid.
    rotation : 2-D array
        Rotation matrix for hyperellipsoid semi-axes (only for 2D or 3D data).

    References
    ----------
    .. [1] http://www.jstor.org/stable/2282774
    .. [2] http://en.wikipedia.org/wiki/Principal_component_analysis
    .. [3] http://en.wikipedia.org/wiki/Singular_value_decomposition
    .. [4] http://en.wikipedia.org/wiki/Volume_of_an_n-ball

    Examples
    --------
    >>> from hyperellipsoid import hyperellipsoid
    >>> y = np.cumsum(np.random.randn(3000)) / 50
    >>> x = np.cumsum(np.random.randn(3000)) / 100
    >>> area, axes, angles, center, R = hyperellipsoid(x, y, units='cm')
    >>> print('Area =', area)
    >>> print('Semi-axes =', axes)
    >>> print('Angles =', angles)
    >>> print('Center =', center)
    >>> print('Rotation matrix =\n', R)

    >>> P = np.random.randn(1000, 3)
    >>> P[:, 2] = P[:, 2] + P[:, 1]*.5
    >>> P[:, 1] = P[:, 1] + P[:, 0]*.5
    >>> volume, axes, angles, center, R = hyperellipsoid(P, units='cm')
    """

    from scipy.stats import f as F
    from scipy.special import gamma

    P = np.array(P, ndmin=2, dtype=float)
    if P.shape[0] == 1:
        P = P.T
    if y is not None:
        y = np.array(y, copy=False, ndmin=2, dtype=float)
        if y.shape[0] == 1:
            y = y.T
        P = np.concatenate((P, y), axis=1)
    if z is not None:
        z = np.array(z, copy=False, ndmin=2, dtype=float)
        if z.shape[0] == 1:
            z = z.T
        P = np.concatenate((P, z), axis=1)
    # covariance matrix
    cov = np.cov(P, rowvar=0)
    # singular value decomposition
    U, s, Vt = np.linalg.svd(cov)
    p, n = s.size, P.shape[0]
    # F percent point function
    fppf = F.ppf(pvalue, p, n-p)*(n-1)*p*(n+1)/n/(n-p)
    # semi-axes (largest first)
    saxes = np.sqrt(s*fppf)
    hypervolume = np.pi**(p/2)/gamma(p/2+1)*np.prod(saxes)
    # rotation matrix
    if p == 2 or p == 3:
        R = Vt
        if s.size == 2:
            angles = np.array([np.rad2deg(np.arctan2(R[1, 0], R[0, 0])),
                               90-np.rad2deg(np.arctan2(R[1, 0], -R[0, 0]))])
        else:
            angles = rotXYZ(R, unit='deg')
        # centroid of the hyperellipsoid
        center = np.mean(P, axis=0)
    else:
        R, angles = None, None

    if show and (p == 2 or p == 3):
        _plot(P, hypervolume, saxes, center, R, pvalue, units, ax)

    return hypervolume, saxes, angles, center, R


def _plot(P, hypervolume, saxes, center, R, pvalue, units, ax):
    """Plot results of the hyperellipsoid function, see its help."""

    try:
        import matplotlib.pyplot as plt
    except ImportError:
        print('matplotlib is not available.')
    else:
        # code based on https://github.com/minillinim/ellipsoid:
        # parametric equations
        u = np.linspace(0, 2*np.pi, 100)
        if saxes.size == 2:
            x = saxes[0]*np.cos(u)
            y = saxes[1]*np.sin(u)
            # rotate data
            for i in range(len(x)):
                [x[i], y[i]] = np.dot([x[i], y[i]], R) + center
        else:
            v = np.linspace(0, np.pi, 100)
            x = saxes[0]*np.outer(np.cos(u), np.sin(v))
            y = saxes[1]*np.outer(np.sin(u), np.sin(v))
            z = saxes[2]*np.outer(np.ones_like(u), np.cos(v))
            # rotate data
            for i in range(len(x)):
                for j in range(len(x)):
                    [x[i,j],y[i,j],z[i,j]] = np.dot([x[i,j],y[i,j],z[i,j]], R) + center

        if saxes.size == 2:
            if ax is None:
                fig, ax = plt.subplots(1, 1, figsize=(5, 5))
            # plot raw data
            ax.plot(P[:, 0], P[:, 1], '.-', color=[0, 0, 1, .5])
            # plot ellipse
            ax.plot(x, y, color=[0, 1, 0, 1], linewidth=2)
            # plot axes
            for i in range(saxes.size):
                # rotate axes
                a = np.dot(np.diag(saxes)[i], R).reshape(2, 1)
                # points for the axes extremities
                a = np.dot(a, np.array([-1, 1], ndmin=2))+center.reshape(2, 1)
                ax.plot(a[0], a[1], color=[1, 0, 0, .6], linewidth=2)
                ax.text(a[0, 1], a[1, 1], '%d' % (i + 1),
                        fontsize=20, color='r')
            plt.axis('equal')
            plt.grid()
            title = r'Prediction ellipse (p=%4.2f): Area=' % pvalue
            if units is not None:
                units2 = ' [%s]' % units
                units = units + r'$^2$'
                title = title + r'%.2f %s' % (hypervolume, units)
            else:
                units2 = ''
                title = title + r'%.2f' % hypervolume
        else:
            from mpl_toolkits.mplot3d import Axes3D
            if ax is None:
                fig = plt.figure(figsize=(7, 7))
                ax = fig.add_axes([0, 0, 1, 1], projection='3d')
            ax.view_init(20, 30)
            # plot raw data
            ax.plot(P[:, 0], P[:, 1], P[:, 2], '.-', color=[0, 0, 1, .4])
            # plot ellipsoid
            ax.plot_surface(x, y, z, rstride=5, cstride=5, color=[0, 1, 0, .1],
                            linewidth=1, edgecolor=[.1, .9, .1, .4])
            # ax.plot_wireframe(x, y, z, color=[0, 1, 0, .5], linewidth=1)
            #                  rstride=3, cstride=3, edgecolor=[0, 1, 0, .5])
            # plot axes
            for i in range(saxes.size):
                # rotate axes
                a = np.dot(np.diag(saxes)[i], R).reshape(3, 1)
                # points for the axes extremities
                a = np.dot(a, np.array([-1, 1], ndmin=2))+center.reshape(3, 1)
                ax.plot(a[0], a[1], a[2], color=[1, 0, 0, .6], linewidth=2)
                ax.text(a[0, 1], a[1, 1], a[2, 1], '%d' % (i+1),
                        fontsize=20, color='r')
            lims = [np.min([P.min(), x.min(), y.min(), z.min()]),
                    np.max([P.max(), x.max(), y.max(), z.max()])]
            ax.set_xlim(lims)
            ax.set_ylim(lims)
            ax.set_zlim(lims)
            title = r'Prediction ellipsoid (p=%4.2f): Volume=' % pvalue
            if units is not None:
                units2 = ' [%s]' % units
                units = units + r'$^3$'
                title = title + r'%.2f %s' % (hypervolume, units)
            else:
                units2 = ''
                title = title + r'%.2f' % hypervolume
            ax.set_zlabel('Z' + units2, fontsize=18)

        ax.set_xlabel('X' + units2, fontsize=18)
        ax.set_ylabel('Y' + units2, fontsize=18)
        plt.title(title)
        plt.show()

        return ax


def rotXYZ(R, unit='deg'):
    """ Compute Euler angles from matrix R using XYZ sequence."""

    angles = np.zeros(3)
    angles[0] = np.arctan2(R[2, 1], R[2, 2])
    angles[1] = np.arctan2(-R[2, 0], np.sqrt(R[0, 0]**2 + R[1, 0]**2))
    angles[2] = np.arctan2(R[1, 0], R[0, 0])

    if unit[:3].lower() == 'deg':  # convert from rad to degree
        angles = np.rad2deg(angles)

    return angles




#sway area
def get_area(COM_data):
    area, axes, angles, center, R = hyperellipsoid((COM_data)[:, 1], (COM_data)[:, 0], units='m')
    return area
Areas_gc=[]
for i in COM_files_gc:
    Areas_gc.append(get_area(i))


#Range AP
def rang(COM, axis_id):
    rang = np.max(COM, axis=0) - np.min(COM, axis=0)  # range (maximum - minimum)
    return rang[axis_id]

def get_rang_AP(COM_data):
    range_ap= rang(COM_data, 0)
    return range_ap

range_AP_gc=[]
for i in COM_files_gc:
    range_AP_gc.append(get_rang_AP(i))

#Range ML
def get_rang_ML(COM_data):
    range_ml= rang(COM_data, 1)
    return range_ml

range_ML_gc=[]
for i in COM_files_gc:
    range_ML_gc.append(get_rang_ML(i))




#Sway Length AP
def sway_len(COM, axis_id):
    tpath = np.sum(np.abs(np.diff(COM, axis=0)), axis=0) # total path (length of the COP displacement)
    return tpath[axis_id]

def get_SL_AP(COM_data):
    sl_ap= sway_len(COM_data, 0)
    return sl_ap

SL_AP_gc=[]
for i in COM_files_gc:
    SL_AP_gc.append(get_SL_AP(i))

#Sl ML
def get_SL_ML(COM_data):
    sl_ml= sway_len(COM_data, 1)
    return sl_ml

SL_ML_gc=[]
for i in COM_files_gc:
    SL_ML_gc.append(get_SL_ML(i))


# Velocity
def get_Velocity(disp_array, time_array):
    '''
        Calculate the velocity of COM
    '''
    # disp_array= df.values
    x= (np.abs(np.diff(disp_array[:, 0])))/ np.diff(time_array)#ap
    y= (np.abs(np.diff(disp_array[:, 1])))/ np.diff(time_array)#ml
    df = pd.DataFrame({ 'Vel_COM_AP':x,'Vel_COM_ML':y})
    return df.values
    # Data= Data.apply(lambda x: x/1000)
velocities_gc=[]
for i in COM_files_gc:
    velocities_gc.append(get_Velocity(i, Time_gc))
# Mean Velocity
def get_mean_Velocity(disp_array, time_array):
    '''
        Calculate the mean velocity of COM
    '''
    x= np.sum(np.abs(np.diff(disp_array[:, 0])))/ np.sum(np.diff(time_array))#ap
    y= np.sum(np.abs(np.diff(disp_array[:, 1])))/ np.sum(np.diff(time_array))#ml
    # df = pd.DataFrame({ 'Mean_Vel_COM_AP':x,'Mean_Vel_COM_ML':y})
    return np.array([x,y])
mean_velocities_gc=[]
for i in COM_files_gc:
    mean_velocities_gc.append(get_mean_Velocity(i, Time_gc))



# acceleration
Time_gc= np.linspace(0,30,len(velocities_gc[0]))#update time
def get_acceleration(vel_array, time_array):
    '''
        Calculate the acceleration of COM
    '''
    # vel_array= df.values
    x= (np.diff(vel_array[:, 0]))/ np.diff(time_array)#ap
    y= (np.diff(vel_array[:, 1]))/ np.diff(time_array)#ml
    df = pd.DataFrame({ 'Acc_COM_AP':x,'Acc_COM_ML':y})
    return df.values
    # Data= Data.apply(lambda x: x/1000)
acc_gc=[]
for i in velocities_gc:
    acc_gc.append(get_acceleration(i, Time_gc))
# Mean acceleration
def get_mean_acceleration(vel_array, time_array):
    '''
        Calculate the mean acceleration of COM
    '''
    x= np.sum(np.abs(np.diff(vel_array[:, 0])))/ np.sum(np.diff(time_array))#ap
    y= np.sum(np.abs(np.diff(vel_array[:, 1])))/ np.sum(np.diff(time_array))#ml
    # df = pd.DataFrame({ 'Mean_Vel_COM_AP':x,'Mean_Vel_COM_ML':y})
    return np.array([x,y])
mean_acc_gc=[]
for i in velocities_gc:
    mean_acc_gc.append(get_mean_acceleration(i, Time_gc))
def get_rms_acceleration(mean_acc_list):
    '''
        Calculate the rms acceleration of COM
    '''
    rms_acceleration=[]
    for i in mean_acc_list:
        rms_acceleration.append(np.sqrt(i))
    return rms_acceleration
rms_acc_gc= get_rms_acceleration(mean_acc_gc)







data_gc = []
for i in range(len(COM_files_gc)):
    outcomes=[Areas_gc[i], range_AP_gc[i], range_ML_gc[i], SL_AP_gc[i], SL_ML_gc[i], mean_velocities_gc[i][0], mean_velocities_gc[i][1]
    , (rms_acc_gc[i]/9.8)[0], (rms_acc_gc[i]/9.8)[1]]#convert in g-unit
    data_gc.append(outcomes)

df_gc = pd.DataFrame(data_gc, columns=['Area (m^2)_gc', 'Range AP (m)_gc', 'Range ML (m)_gc', "Sway Length AP (m)_gc", "Sway Length ML (m)_gc",
'Mean Velocity AP (m/s)_gc', 'Mean Velocity ML (m/s)_gc', 'RMS Acc AP (g)_gc', 'RMS Acc ML (g)_gc'])




In [6]:
#_go

# Define relative path to folder containing the text files

# files_folder = "/Users/cyrilleetude/Desktop/Internship_deliverables/datasets/Static/Healthy_static/Csv/processed"
files_folder_go = "/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Processed_go"#too lazy to change everything so let's keep the processed_file name for the raw ones and only change chaen saving
# files_folder= "/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/FallersCOMraw"
processed_files_go = []

# Create a dataframe list by using a list comprehension

processed_files_go = [pd.read_csv(file) for file in glob.glob(os.path.join(files_folder_go ,"*.csv"))]


Time_go= np.linspace(0,30,len(processed_files_go[0]))
def clean_df(df):
    '''
        Clean a df by removing the firs row
    '''
    return df.loc[:, ['COM_AP', 'COM_ML']]
CleanProcessedFiles_go= []
for i in processed_files_go:
    CleanProcessedFiles_go.append(clean_df(i))


def get_com(processed_df):
    # processed_df['time']= [i/100 for i in range(len(processed_df))]
    # freq= np.mean(1/np.diff(Time.to_list()))
    COM= processed_df.loc[:, ['COM_AP', 'COM_ML']]
    # COM= (COM.values)*10 #convert cm to mm no keep it in meters
    COM = detrend(COM, axis=0, type='linear')
    return COM

COM_files_go=[]
for i in CleanProcessedFiles_go:
    COM_files_go.append(get_com(i))



#function
def hyperellipsoid(P, y=None, z=None, pvalue=.95, units=None, show=False, ax=None):
    """
    Prediction hyperellipsoid for multivariate data.

    The hyperellipsoid is a prediction interval for a sample of a multivariate
    random variable and is such that there is pvalue*100% of probability that a
    new observation will be contained inside the hyperellipsoid [1]_.  
    The hyperellipsoid is also a tolerance region such that the average or
    expected value of the proportion of the population contained in this region
    is exactly pvalue*100% (called Type 2 tolerance region by Chew (1966) [1]_).

    The directions and lengths of the semi-axes are found, respectively, as the
    eigenvectors and eigenvalues of the covariance matrix of the data using
    the concept of principal components analysis (PCA) [2]_ or singular value
    decomposition (SVD) [3]_ and the length of the semi-axes are adjusted to
    account for the necessary prediction probability.

    The volume of the hyperellipsoid is calculated with the same equation for
    the volume of a n-dimensional ball [4]_ with the radius replaced by the
    semi-axes of the hyperellipsoid.

    This function calculates the prediction hyperellipsoid for the data,
    which is considered a (finite) sample of a multivariate random variable
    with normal distribution (i.e., the F distribution is used and not
    the approximation by the chi-square distribution).

    Parameters
    ----------
    P : 1-D or 2-D array_like
        For a 1-D array, P is the abscissa values of the [x,y] or [x,y,z] data.
        For a 2-D array, P is the joined values of the multivariate data.
        The shape of the 2-D array should be (n, p) where n is the number of
        observations (rows) and p the number of dimensions (columns).
    y : 1-D array_like, optional (default = None)
        Ordinate values of the [x, y] or [x, y, z] data.
    z : 1-D array_like, optional (default = None)
        Ordinate values of the [x, y] or [x, y, z] data.
    pvalue : float, optional (default = .95)
        Desired prediction probability of the hyperellipsoid.
    units : str, optional (default = None)
        Units of the input data.
    show : bool, optional (default = True)
        True (1) plots data in a matplotlib figure, False (0) to not plot.
        Only the results for p=2 (ellipse) or p=3 (ellipsoid) will be plotted.
    ax : a matplotlib.axes.Axes instance (default = None)

    Returns
    -------
    hypervolume : float
        Hypervolume (e.g., area of the ellipse or volume of the ellipsoid).
    axes : 1-D array
        Lengths of the semi-axes hyperellipsoid (largest first).
    angles : 1-D array
        Angles of the semi-axes hyperellipsoid (only for 2D or 3D data).
        For the ellipsoid (3D data), the angles are the Euler angles
        calculated in the XYZ sequence.
    center : 1-D array
        Centroid of the hyperellipsoid.
    rotation : 2-D array
        Rotation matrix for hyperellipsoid semi-axes (only for 2D or 3D data).

    References
    ----------
    .. [1] http://www.jstor.org/stable/2282774
    .. [2] http://en.wikipedia.org/wiki/Principal_component_analysis
    .. [3] http://en.wikipedia.org/wiki/Singular_value_decomposition
    .. [4] http://en.wikipedia.org/wiki/Volume_of_an_n-ball

    Examples
    --------
    >>> from hyperellipsoid import hyperellipsoid
    >>> y = np.cumsum(np.random.randn(3000)) / 50
    >>> x = np.cumsum(np.random.randn(3000)) / 100
    >>> area, axes, angles, center, R = hyperellipsoid(x, y, units='cm')
    >>> print('Area =', area)
    >>> print('Semi-axes =', axes)
    >>> print('Angles =', angles)
    >>> print('Center =', center)
    >>> print('Rotation matrix =\n', R)

    >>> P = np.random.randn(1000, 3)
    >>> P[:, 2] = P[:, 2] + P[:, 1]*.5
    >>> P[:, 1] = P[:, 1] + P[:, 0]*.5
    >>> volume, axes, angles, center, R = hyperellipsoid(P, units='cm')
    """

    from scipy.stats import f as F
    from scipy.special import gamma

    P = np.array(P, ndmin=2, dtype=float)
    if P.shape[0] == 1:
        P = P.T
    if y is not None:
        y = np.array(y, copy=False, ndmin=2, dtype=float)
        if y.shape[0] == 1:
            y = y.T
        P = np.concatenate((P, y), axis=1)
    if z is not None:
        z = np.array(z, copy=False, ndmin=2, dtype=float)
        if z.shape[0] == 1:
            z = z.T
        P = np.concatenate((P, z), axis=1)
    # covariance matrix
    cov = np.cov(P, rowvar=0)
    # singular value decomposition
    U, s, Vt = np.linalg.svd(cov)
    p, n = s.size, P.shape[0]
    # F percent point function
    fppf = F.ppf(pvalue, p, n-p)*(n-1)*p*(n+1)/n/(n-p)
    # semi-axes (largest first)
    saxes = np.sqrt(s*fppf)
    hypervolume = np.pi**(p/2)/gamma(p/2+1)*np.prod(saxes)
    # rotation matrix
    if p == 2 or p == 3:
        R = Vt
        if s.size == 2:
            angles = np.array([np.rad2deg(np.arctan2(R[1, 0], R[0, 0])),
                               90-np.rad2deg(np.arctan2(R[1, 0], -R[0, 0]))])
        else:
            angles = rotXYZ(R, unit='deg')
        # centroid of the hyperellipsoid
        center = np.mean(P, axis=0)
    else:
        R, angles = None, None

    if show and (p == 2 or p == 3):
        _plot(P, hypervolume, saxes, center, R, pvalue, units, ax)

    return hypervolume, saxes, angles, center, R


def _plot(P, hypervolume, saxes, center, R, pvalue, units, ax):
    """Plot results of the hyperellipsoid function, see its help."""

    try:
        import matplotlib.pyplot as plt
    except ImportError:
        print('matplotlib is not available.')
    else:
        # code based on https://github.com/minillinim/ellipsoid:
        # parametric equations
        u = np.linspace(0, 2*np.pi, 100)
        if saxes.size == 2:
            x = saxes[0]*np.cos(u)
            y = saxes[1]*np.sin(u)
            # rotate data
            for i in range(len(x)):
                [x[i], y[i]] = np.dot([x[i], y[i]], R) + center
        else:
            v = np.linspace(0, np.pi, 100)
            x = saxes[0]*np.outer(np.cos(u), np.sin(v))
            y = saxes[1]*np.outer(np.sin(u), np.sin(v))
            z = saxes[2]*np.outer(np.ones_like(u), np.cos(v))
            # rotate data
            for i in range(len(x)):
                for j in range(len(x)):
                    [x[i,j],y[i,j],z[i,j]] = np.dot([x[i,j],y[i,j],z[i,j]], R) + center

        if saxes.size == 2:
            if ax is None:
                fig, ax = plt.subplots(1, 1, figsize=(5, 5))
            # plot raw data
            ax.plot(P[:, 0], P[:, 1], '.-', color=[0, 0, 1, .5])
            # plot ellipse
            ax.plot(x, y, color=[0, 1, 0, 1], linewidth=2)
            # plot axes
            for i in range(saxes.size):
                # rotate axes
                a = np.dot(np.diag(saxes)[i], R).reshape(2, 1)
                # points for the axes extremities
                a = np.dot(a, np.array([-1, 1], ndmin=2))+center.reshape(2, 1)
                ax.plot(a[0], a[1], color=[1, 0, 0, .6], linewidth=2)
                ax.text(a[0, 1], a[1, 1], '%d' % (i + 1),
                        fontsize=20, color='r')
            plt.axis('equal')
            plt.grid()
            title = r'Prediction ellipse (p=%4.2f): Area=' % pvalue
            if units is not None:
                units2 = ' [%s]' % units
                units = units + r'$^2$'
                title = title + r'%.2f %s' % (hypervolume, units)
            else:
                units2 = ''
                title = title + r'%.2f' % hypervolume
        else:
            from mpl_toolkits.mplot3d import Axes3D
            if ax is None:
                fig = plt.figure(figsize=(7, 7))
                ax = fig.add_axes([0, 0, 1, 1], projection='3d')
            ax.view_init(20, 30)
            # plot raw data
            ax.plot(P[:, 0], P[:, 1], P[:, 2], '.-', color=[0, 0, 1, .4])
            # plot ellipsoid
            ax.plot_surface(x, y, z, rstride=5, cstride=5, color=[0, 1, 0, .1],
                            linewidth=1, edgecolor=[.1, .9, .1, .4])
            # ax.plot_wireframe(x, y, z, color=[0, 1, 0, .5], linewidth=1)
            #                  rstride=3, cstride=3, edgecolor=[0, 1, 0, .5])
            # plot axes
            for i in range(saxes.size):
                # rotate axes
                a = np.dot(np.diag(saxes)[i], R).reshape(3, 1)
                # points for the axes extremities
                a = np.dot(a, np.array([-1, 1], ndmin=2))+center.reshape(3, 1)
                ax.plot(a[0], a[1], a[2], color=[1, 0, 0, .6], linewidth=2)
                ax.text(a[0, 1], a[1, 1], a[2, 1], '%d' % (i+1),
                        fontsize=20, color='r')
            lims = [np.min([P.min(), x.min(), y.min(), z.min()]),
                    np.max([P.max(), x.max(), y.max(), z.max()])]
            ax.set_xlim(lims)
            ax.set_ylim(lims)
            ax.set_zlim(lims)
            title = r'Prediction ellipsoid (p=%4.2f): Volume=' % pvalue
            if units is not None:
                units2 = ' [%s]' % units
                units = units + r'$^3$'
                title = title + r'%.2f %s' % (hypervolume, units)
            else:
                units2 = ''
                title = title + r'%.2f' % hypervolume
            ax.set_zlabel('Z' + units2, fontsize=18)

        ax.set_xlabel('X' + units2, fontsize=18)
        ax.set_ylabel('Y' + units2, fontsize=18)
        plt.title(title)
        plt.show()

        return ax


def rotXYZ(R, unit='deg'):
    """ Compute Euler angles from matrix R using XYZ sequence."""

    angles = np.zeros(3)
    angles[0] = np.arctan2(R[2, 1], R[2, 2])
    angles[1] = np.arctan2(-R[2, 0], np.sqrt(R[0, 0]**2 + R[1, 0]**2))
    angles[2] = np.arctan2(R[1, 0], R[0, 0])

    if unit[:3].lower() == 'deg':  # convert from rad to degree
        angles = np.rad2deg(angles)

    return angles




#sway area
def get_area(COM_data):
    area, axes, angles, center, R = hyperellipsoid((COM_data)[:, 1], (COM_data)[:, 0], units='m')
    return area
Areas_go=[]
for i in COM_files_go:
    Areas_go.append(get_area(i))


#Range AP
def rang(COM, axis_id):
    rang = np.max(COM, axis=0) - np.min(COM, axis=0)  # range (maximum - minimum)
    return rang[axis_id]

def get_rang_AP(COM_data):
    range_ap= rang(COM_data, 0)
    return range_ap

range_AP_go=[]
for i in COM_files_go:
    range_AP_go.append(get_rang_AP(i))

#Range ML
def get_rang_ML(COM_data):
    range_ml= rang(COM_data, 1)
    return range_ml

range_ML_go=[]
for i in COM_files_go:
    range_ML_go.append(get_rang_ML(i))




#Sway Length AP
def sway_len(COM, axis_id):
    tpath = np.sum(np.abs(np.diff(COM, axis=0)), axis=0) # total path (length of the COP displacement)
    return tpath[axis_id]

def get_SL_AP(COM_data):
    sl_ap= sway_len(COM_data, 0)
    return sl_ap

SL_AP_go=[]
for i in COM_files_go:
    SL_AP_go.append(get_SL_AP(i))

#Sl ML
def get_SL_ML(COM_data):
    sl_ml= sway_len(COM_data, 1)
    return sl_ml

SL_ML_go=[]
for i in COM_files_go:
    SL_ML_go.append(get_SL_ML(i))


# Velocity
def get_Velocity(disp_array, time_array):
    '''
        Calculate the velocity of COM
    '''
    # disp_array= df.values
    x= (np.abs(np.diff(disp_array[:, 0])))/ np.diff(time_array)#ap
    y= (np.abs(np.diff(disp_array[:, 1])))/ np.diff(time_array)#ml
    df = pd.DataFrame({ 'Vel_COM_AP':x,'Vel_COM_ML':y})
    return df.values
    # Data= Data.apply(lambda x: x/1000)
velocities_go=[]
for i in COM_files_go:
    velocities_go.append(get_Velocity(i, Time_go))
# Mean Velocity
def get_mean_Velocity(disp_array, time_array):
    '''
        Calculate the mean velocity of COM
    '''
    x= np.sum(np.abs(np.diff(disp_array[:, 0])))/ np.sum(np.diff(time_array))#ap
    y= np.sum(np.abs(np.diff(disp_array[:, 1])))/ np.sum(np.diff(time_array))#ml
    # df = pd.DataFrame({ 'Mean_Vel_COM_AP':x,'Mean_Vel_COM_ML':y})
    return np.array([x,y])
mean_velocities_go=[]
for i in COM_files_go:
    mean_velocities_go.append(get_mean_Velocity(i, Time_go))



# acceleration
Time_go= np.linspace(0,30,len(velocities_go[0]))#update time
def get_acceleration(vel_array, time_array):
    '''
        Calculate the acceleration of COM
    '''
    # vel_array= df.values
    x= (np.diff(vel_array[:, 0]))/ np.diff(time_array)#ap
    y= (np.diff(vel_array[:, 1]))/ np.diff(time_array)#ml
    df = pd.DataFrame({ 'Acc_COM_AP':x,'Acc_COM_ML':y})
    return df.values
    # Data= Data.apply(lambda x: x/1000)
acc_go=[]
for i in velocities_go:
    acc_go.append(get_acceleration(i, Time_go))
# Mean acceleration
def get_mean_acceleration(vel_array, time_array):
    '''
        Calculate the mean acceleration of COM
    '''
    x= np.sum(np.abs(np.diff(vel_array[:, 0])))/ np.sum(np.diff(time_array))#ap
    y= np.sum(np.abs(np.diff(vel_array[:, 1])))/ np.sum(np.diff(time_array))#ml
    # df = pd.DataFrame({ 'Mean_Vel_COM_AP':x,'Mean_Vel_COM_ML':y})
    return np.array([x,y])
mean_acc_go=[]
for i in velocities_go:
    mean_acc_go.append(get_mean_acceleration(i, Time_go))
def get_rms_acceleration(mean_acc_list):
    '''
        Calculate the rms acceleration of COM
    '''
    rms_acceleration=[]
    for i in mean_acc_list:
        rms_acceleration.append(np.sqrt(i))
    return rms_acceleration
rms_acc_go= get_rms_acceleration(mean_acc_go)







data_go = []
for i in range(len(COM_files_go)):
    outcomes=[Areas_go[i], range_AP_go[i], range_ML_go[i], SL_AP_go[i], SL_ML_go[i], mean_velocities_go[i][0], mean_velocities_go[i][1]
    , (rms_acc_go[i]/9.8)[0], (rms_acc_go[i]/9.8)[1]]#convert in g-unit
    data_go.append(outcomes)

df_go = pd.DataFrame(data_go, columns=['Area (m^2)_go', 'Range AP (m)_go', 'Range ML (m)_go', "Sway Length AP (m)_go", "Sway Length ML (m)_go",
'Mean Velocity AP (m/s)_go', 'Mean Velocity ML (m/s)_go', 'RMS Acc AP (g)_go', 'RMS Acc ML (g)_go'])




In [7]:
other_features = pd.read_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Other_FEatures/faller_features.csv')
other_features.drop(other_features.columns[[0]], axis=1, inplace=True)
df_fallers = pd.concat([df_fc, df_fo, df_gc, df_go, other_features], axis=1)
df_fallers.to_csv('/Users/cyrilleetude/Desktop/Bureau - MacBook Air de Cyrille/Semester_Project_Cyrille_2022/MachineLEARNING/Fallers_Data_Analyzed/Fallers_data_analyzed.csv')

In [14]:
processed_files_go[0].COM_AP.to_list()[:10] , processed_files_go[0].COM_ML.to_list()[:10] 

([-0.0020981057139296,
  -0.0021452990890011,
  -0.002192778771767,
  -0.0022405761747172,
  -0.0022887289161361,
  -0.0023372888277896,
  -0.002386327378805,
  -0.0024359341767124,
  -0.0024862102060233,
  -0.0025372549093793],
 [0.0093264354783519,
  0.0093069087973385,
  0.0092873315816097,
  0.0092677738346731,
  0.0092483289346212,
  0.0092291040117104,
  0.0092102074137082,
  0.0091917365456323,
  0.0091737693080236,
  0.0091563622735829])